In [ ]:
import h5py    
import numpy as np    
from astropy.table import Table
from matplotlib import pyplot as plt

from splashback.cluster import cluster_sample
from astropy import units as u
from splashback.profile import DK14, rho, rho_infall, rho_Ein, f_trans, NFW, Mvir_to_M200m
import scipy.interpolate
import scipy.integrate


%matplotlib inline
plt.rc('font', size=20)

***Density profiles and cluster selection***
------------

In [ ]:
file_data = h5py.File("data/sim_clusters.hdf5",'r+')   
M200m = np.array([14.24, 14.15, 14.15, 14.31, 14.34, 14.35, 14.52, 14.53, 14.49, 14.63, 14.55, 14.63, 14.71, 14.72, 14.79, 14.83, 14.88, 14.96, 15.09, 15.23, 15.19, 15.28, 15.34, 15.51])
M200m = 10**M200m
M500c = 10**np.array(file_data['M500c'])

idxs = [16, 17, 18, 19, 20,21,22,23]

print 'mean M500c of idxs:', M500c[idxs].mean()/1e14, " 1e14 Msun"
print "mean M200m of idxs", M200m[idxs].mean()/1e14, " 1e14 Msun"
print "CE IDS of idxs:", file_data['CE_Number'][idxs]

plt.loglog(file_data['BinLims'], file_data['DensityProfile2D'][idxs, :, 1, 4].T)
plt.loglog(file_data['BinLims'], file_data['DensityProfile2D'][idxs, :, 2, 4].T)
plt.loglog(file_data['BinLims'], file_data['DensityProfile2D'][idxs, :, 0, 4].T)
plt.gca().set_xlabel('R')
plt.gca().set_ylabel(r'$\Sigma$')
plt.show()

***Compute lensing signal***
--------------

In [ ]:
# Define rbin for the interpolation of the projected profile
bin_edges = np.concatenate([[0], file_data['BinLims']])
rmin = bin_edges[:-1]
rmax = bin_edges[1:]
rbin = (rmax**3-rmin**3)/(rmax**2-rmin**2)*2./3.
drbin = rmax-rmin


# Define Rbin for the final Delta Sigma
bin_edges = np.geomspace(.2, 9, 11) 
rmin = bin_edges[:-1]
rmax = bin_edges[1:]
Rbin = (rmax**3-rmin**3)/(rmax**2-rmin**2)*2./3.
delta_sigma_dm = np.zeros([len(idxs)*3, len(Rbin)])
M500 = np.zeros(len(idxs)*3)


#Integrate + stack
i=0
for n_cluster in idxs:
    for n_direction in xrange(3):
        func = scipy.interpolate.interp1d(rbin, \
            (file_data['DensityProfile2D'][n_cluster, :, n_direction, 4]),  bounds_error=False, fill_value=np.nan)
        
        for j in xrange(len(Rbin)):
            sig = file_data['DensityProfile2D'][n_cluster, :, n_direction, 4]
            integral = (rbin[rbin<=Rbin[j]]*sig[rbin<=Rbin[j]]* drbin[rbin<=Rbin[j]] ).sum()
            delta_sigma_dm[i, j] = 2.*integral/Rbin[j]/Rbin[j] - func(Rbin[j])
        
        M500[i] = M500c[n_cluster]
        i+=1

        
#stack + integrate
sum_sigma = np.zeros(len(rbin))
delta_sigma_2 = np.zeros(len(rbin))
for n_cluster in idxs:
    #for n_direction in xrange(3):
    n_direction = 1
    sum_sigma += (file_data['DensityProfile2D'][n_cluster, :, n_direction, 4])

func = scipy.interpolate.interp1d(rbin, sum_sigma,  bounds_error=False, fill_value=np.nan)

for j in xrange(len(rbin)):
    integral = (rbin[rbin<=rbin[j]]*sum_sigma[rbin<=rbin[j]]* drbin[rbin<=rbin[j]] ).sum()
    delta_sigma_2[j] = 2.*integral/rbin[j]/rbin[j] - func(rbin[j])    

delta_sigma_2 = delta_sigma_2/len(idxs)



plt.figure(figsize=(10, 7))

#Data
CCCP = cluster_sample('data/CCCPhighm.fits', 'data/source/')
bin_edges2 = np.geomspace(.2, 9, 11)*u.Mpc
CCCP.stack_ESD(bin_edges2, comoving=True, mscaling=False, weighted=False, cosmic_noise='output/lss_covariance/')
x = CCCP.rbin.value
y = CCCP.ESD.value
yerr = CCCP.ESDerr.value
plt.errorbar(x, y, yerr=yerr, label='measured', fmt='o', c='red', alpha=1, capsize=10)


for i in xrange(len(idxs)*3):
    plt.plot(Rbin, delta_sigma_dm[i, :], alpha=0.2, c='0.4')

#Simulation
y_sim = delta_sigma_dm.mean(0)
y_sim_err = delta_sigma_dm.std(0)
plt.errorbar(Rbin, y_sim, yerr=y_sim_err, label='sim individual stack')
plt.plot(rbin, delta_sigma_2, label='sim full stack')

plt.gca().set_xlim([1e-1, 10])
plt.gca().set_ylim([1e13, 1e15])
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')
plt.legend()
plt.show()



np.save('output/profiles/simulation_ds.npy', delta_sigma_2)
np.save('output/profiles/simulation_dsr.npy', rbin)

**Fit profile to NFW at z=0, to test:**
------------

In [ ]:
# FIT TO SIMULATION

z_pivot = 0.
y = y_sim
Covinv = np.diag( (1./y_sim_err)**2. )


Mvirarray, carray = np.meshgrid(np.linspace(1., 30., 100), np.geomspace(2., 10., 100))
Mvirarray = Mvirarray.flatten()
carray = carray.flatten()

L = np.zeros(len(Mvirarray))
for i in xrange(len(Mvirarray)):
    Mvir = Mvirarray[i]*1e14*u.Msun
    c= carray[i]
    z = z_pivot
    
    prof = NFW(Mvir, z, c)
    model = prof.D_Sigma(Rbin)
    L[i] = (-0.5*np.matrix(y[:]-model[:])*Covinv*np.matrix(y[:]-model[:]).T).sum()

print "Mvir, z, c: ", Mvirarray[L.argmax()], z_pivot, carray[L.argmax()]#, Mvirarray[L.argmax()]
M200plot, r200plot =  Mvir_to_M200m(Mvirarray[L.argmax()]*1e14*u.Msun, z=z_pivot, also_r=True, c=carray[L.argmax()])
print "M200m, r200m stack at: ", Mvir_to_M200m(Mvirarray[L.argmax()]*1e14*u.Msun, z=z_pivot, also_r=True)
print L.max()


***Compute logarithmic slope***
===========

In [ ]:
from scipy.signal import savgol_filter


sum_rho = 0*np.copy(sum_sigma)
for n_cluster in idxs:
        sum_rho += file_data['DensityProfile3D'][n_cluster, :, 4]#/M200m[n_cluster]

        
#sum_rho = savgol_filter(sum_rho, 21, 2)        
        

idx = rbin > 1e-1
y = np.gradient(np.log(sum_rho), np.log(rbin))

'''
x_k = np.linspace(-3., 3., 20)
kernel = np.exp(-0.5*(x_k))
kernel = kernel/kernel.sum()
y_c = np.convolve(y, kernel, mode='same')
plt.plot(rbin, y_c)
'''


y_c = savgol_filter(y, 31, 2)
plt.plot(rbin, y_c)

#plt.plot(rbin, y)
plt.gca().set_xscale('log')
plt.gca().set_xlim([1e-1, 9])
plt.gca().set_ylim([-4, -0.5])


#plt.gca().set_yscale('log')
plt.show()

np.save('output/profiles/simulation_g.npy', y_c)
np.save('output/profiles/simulation_rg.npy', rbin)